In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import statsmodels.api as sm

# Carregando o dataset
arquivo_csv = "insurance.csv"
data_frame = pd.read_csv(arquivo_csv)

# Processamento das variáveis categóricas
label_encoder = LabelEncoder()

# Transformando as variáveis categóricas
for coluna in data_frame.select_dtypes(include='object').columns:
    if coluna != 'region':
        data_frame[coluna] = label_encoder.fit_transform(data_frame[coluna])
    else:
        one_hot_encoder = OneHotEncoder(sparse_output=False)
        colunas_codificadas = one_hot_encoder.fit_transform(data_frame.loc[:, coluna:coluna])
        df_colunas_transformadas = pd.DataFrame(colunas_codificadas, columns=one_hot_encoder.get_feature_names_out([coluna]))
        df_codificado = pd.concat([data_frame, df_colunas_transformadas], axis=1)
        df_codificado.drop([coluna], axis=1, inplace=True)
        data_frame = df_codificado

# Definindo as variáveis preditoras (excluindo 'charges')
var_preditoras = data_frame.drop(['charges'], axis=1)
var_alvo = data_frame['charges']

# Separando os dados em treino e teste (80% treino, 20% teste)
x_train, x_test, y_train, y_test = train_test_split(var_preditoras, var_alvo, test_size=0.2, random_state=0)

# Treinamento do modelo de regressão linear
modelo_reg_multi = LinearRegression()
modelo_reg_multi.fit(x_train, y_train)

# Previsões
y_predicao = modelo_reg_multi.predict(x_test)

# Avaliação do modelo: MAE, RMSE, R^2, R^2 ajustado
mae = metrics.mean_absolute_error(y_test, y_predicao)
rmse = np.sqrt(metrics.mean_squared_error(y_test, y_predicao))
r_sq = metrics.r2_score(y_test, y_predicao)
n = len(y_test)
k = x_test.shape[1]
adjusted_r_squared = 1 - (1 - r_sq) * (n - 1) / (n - k - 1)

# Exibindo os resultados
print(f"MAE: {mae}")
print(f"RMSE: {rmse}")
print(f"R^2: {r_sq}")
print(f"R^2 ajustado: {adjusted_r_squared}")


MAE: 3933.272649405236
RMSE: 5641.626558850189
R^2: 0.7999876970680435
R^2 ajustado: 0.7930105237099521


In [2]:
# Cálculo dos p-valores para todas as variáveis preditoras
var_preditoras2 = sm.add_constant(data_frame.drop(['charges'], axis=1))
estimacao = sm.OLS(data_frame['charges'], var_preditoras2)
estimacao_pvalor = estimacao.fit()

# Mostra o sumário com p-valores
print(estimacao_pvalor.summary())

# Com base nos p-valores, escolhemos a variável com maior p-valor para remover.
# Suponhamos que a variável 'region_southwest' tenha o maior p-valor.

# Removendo a variável com o maior p-valor
var_preditoras_sem_region_southwest = var_preditoras.drop(['region_southwest'], axis=1)

# Separando novamente os dados em treino e teste
x_train2, x_test2, y_train2, y_test2 = train_test_split(var_preditoras_sem_region_southwest, var_alvo, test_size=0.2, random_state=0)

# Re-treinando o modelo
modelo_reg_multi2 = LinearRegression()
modelo_reg_multi2.fit(x_train2, y_train2)

# Fazendo as previsões
y_predicao2 = modelo_reg_multi2.predict(x_test2)

# Avaliando o novo modelo
mae2 = metrics.mean_absolute_error(y_test2, y_predicao2)
rmse2 = np.sqrt(metrics.mean_squared_error(y_test2, y_predicao2))
r_sq2 = metrics.r2_score(y_test2, y_predicao2)
adjusted_r_squared2 = 1 - (1 - r_sq2) * (n - 1) / (n - k - 2)  # Ajuste do R^2 para o novo número de variáveis

# Exibindo os resultados do novo modelo
print(f"MAE após remoção: {mae2}")
print(f"RMSE após remoção: {rmse2}")
print(f"R^2 após remoção: {r_sq2}")
print(f"R^2 ajustado após remoção: {adjusted_r_squared2}")


                            OLS Regression Results                            
Dep. Variable:                charges   R-squared:                       0.751
Model:                            OLS   Adj. R-squared:                  0.749
Method:                 Least Squares   F-statistic:                     500.8
Date:                Sun, 29 Sep 2024   Prob (F-statistic):               0.00
Time:                        21:24:55   Log-Likelihood:                -13548.
No. Observations:                1338   AIC:                         2.711e+04
Df Residuals:                    1329   BIC:                         2.716e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const            -1.002e+04    781.640  

Quando se trata de modelos de regressão, é fácil ficar preso na complexidade das métricas e das variáveis. Mas, a verdade é que, ao tentar ajustar todos os fatores ao mesmo tempo, o progresso pode ser mínimo. Isso acontece porque o nosso objetivo aqui é otimizar um modelo simples e eficiente, não um que envolva o máximo de variáveis. Assim como no desenvolvimento pessoal, onde tentar crescer em todas as áreas ao mesmo tempo leva à estagnação, no ajuste de um modelo, menos pode ser mais.

Ao remover uma variável com baixo impacto, o processo se torna mais leve, os resultados ficam mais claros, e o modelo flui com menos ruído.